In [ ]:
# !pip install opencv-python scikit-learn scikit-image matplotlib spectral keras_tuner vis
# !pip install tensorflow numpy pandas
# !pip install spectral

In [1]:
import tensorflow as tf

In [2]:
from tensorflow import keras as keras
from keras import layers as layers
import numpy as np

In [3]:
import os, timeit
import cv2
from skimage.filters import threshold_otsu
import matplotlib.pyplot as plt
from math import inf as inf
import numpy as np

In [4]:
from spectral.io import envi as envi
from spectral import imshow

In [5]:
from sklearn.decomposition import IncrementalPCA

In [6]:
import keras_tuner as kt
import sys

In [7]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [8]:
from sys import platform
DATA_DIRECTORY = ""
SLASH = ""
if platform == "linux" or platform == "linux2":
    DATA_DIRECTORY = "/home/nitintyagi/wheat data/BULK/"
    SLASH = "/"
elif platform == "win32":
    DATA_DIRECTORY = "D:\mvl\wheat\data\BULK\\"
    SLASH="\\"

In [43]:
#Testing Constants
TESTING = False

#Constants
BAND_NUMBER = 60
FILLED_AREA_RATIO = 0.50
IMAGE_COUNT = int(8/4)
NUM_VARIETIES = 2
NUM_OF_BANDS = 15
FIRST_BAND = 21
LAST_BAND = 149

IMAGE_WIDTH = 30
IMAGE_HEIGHT = 30

NUM_EPOCHS = 40
ACTIVATION_TYPE =  ["relu", "tanh","sigmoid"]
BATCH_SIZE = 2*NUM_VARIETIES

LEARNING_RATE_BASE = 0.0001
MIN_LEARNING_RATE_BASE = LEARNING_RATE_BASE/10

FACTOR = 3
NUM_MODELS = 3

In [44]:
def result_file_name():
    return "NumVar_"+str(NUM_VARIETIES)+"_ImageCount_"+str(IMAGE_COUNT)+"_Factor_"+str(FACTOR)+"_MinLR_"+str(MIN_LEARNING_RATE_BASE)+"_LR_"+str(LEARNING_RATE_BASE)+"_FilledArea_"+str(FILLED_AREA_RATIO)+"_NumOfBands_"+str(NUM_OF_BANDS)+"_FB_"+str(FIRST_BAND)+"_LB_"+str(LAST_BAND)+"_BandNo_"+str(BAND_NUMBER)+"_ImageHeight_"+str(IMAGE_HEIGHT)+"_ImageWidth_"+str(IMAGE_WIDTH)+"_BatchSize_"+str(BATCH_SIZE)


In [45]:
def start_timer():
    print("Testing started")
    return timeit.default_timer()

def end_timer():
    return timeit.default_timer()

def show_time(tic,toc): 
    test_time = toc - tic
    print('Testing time (s) = ' + str(test_time) + '\n')

In [46]:
def exactPathHDR(variety,file):
    return DATA_DIRECTORY+variety+SLASH+file+".bil.hdr"

def exactPathBIL(variety,file):
    return DATA_DIRECTORY+variety+SLASH+file+".bil"

In [47]:
def getROI(img,band_number):
    img_band = img.read_band(band_number)
    threshold = threshold_otsu(img_band)
    roi=[]
    for x in range(img_band.shape[0]):
        a=[]
        for y in range(img_band.shape[1]):
            if img_band[x][y]>threshold:
                a.append(1)
            else:
                a.append(0)
        roi.append(a)
    return roi

In [48]:
#Returns range for x and y from where we have to crop images
def getRangeXandY(img,band_number):
    img_band = img.read_band(band_number)
    roi = getROI(img,band_number)
    xmin = inf
    xmax = 0
    ymin = inf
    ymax = 0
    for x in range(img_band.shape[0]):
        for y in range(img_band.shape[1]):
            if roi[x][y]==1:
                if x<xmin:
                    xmin=x
                if x>xmax:
                    xmax=x
                if y<ymin:
                    ymin=y
                if y>ymax:
                    ymax=y
    return xmin, xmax, ymin, ymax

In [49]:
def getCroppedImage(img,band_number):
    xmin, xmax, ymin, ymax = getRangeXandY(img,band_number)
    new_img = img[xmin:xmax, ymin:ymax, :]
    return new_img    

In [50]:
def getCroppedROI(img,band_number):
    xmin, xmax, ymin, ymax = getRangeXandY(img,band_number)
    roi = np.array(getROI(img,band_number))
    roi = roi[xmin:xmax, ymin:ymax]
    return roi   

In [51]:
def getUsefulImage(img,band_number):
    crop_img = getCroppedImage(img,band_number)
    crop_roi = getCroppedROI(img,band_number)
    for x in range(crop_img.shape[2]):
        band = crop_img[:,:,x]
        crop_img[:,:,x] = band*crop_roi
    return crop_img

In [52]:
data_augmentation = keras.Sequential([
    layers.RandomCrop(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
    layers.RandomRotation(factor=(-0.1, 0.1)),
    layers.RandomZoom(height_factor=(-0.1, 0.1), width_factor=(-0.1,0.1)),
    layers.RandomFlip(mode="horizontal_and_vertical", seed=None)
])

def getAugumentedImage(img,band_number):
    new_img = getUsefulImage(img,band_number)
    augmented_image = data_augmentation(new_img) 
    return augmented_image

def checkAugumentedImage(augmented_image):
    aug_band = augmented_image[:,:,0]
    filled_area_ratio = (np.count_nonzero(aug_band))/(aug_band.shape[0]*aug_band.shape[1])
    if filled_area_ratio > FILLED_AREA_RATIO :
        return True
    else:
        return False

In [53]:
## Dimensional Reduction Method
def DL_Method(HSI, numComponents = NUM_OF_BANDS):
    RHSI = np.reshape(HSI, (-1, HSI.shape[2]))
    n_batches = 10
    inc_pca = IncrementalPCA(n_components=numComponents)
    for X_batch in np.array_split(RHSI, n_batches):
        inc_pca.partial_fit(X_batch)
    X_ipca = inc_pca.transform(RHSI)
    RHSI = np.reshape(X_ipca, (HSI.shape[0],HSI.shape[1], numComponents))
    return RHSI

In [54]:
# List for All varieties
VARIETIES = []
VARIETIES_CODE = {}

for name in os.listdir(DATA_DIRECTORY):
    if (name.endswith(".hdr") or name.endswith(".bil")):
        continue
    VARIETIES_CODE[name] = len(VARIETIES)
    VARIETIES.append(name)
    if len(VARIETIES)==NUM_VARIETIES:
        break

In [55]:
#List for all file names in varities
FILES = []
MAX_FILE_NUM = 4
for x in range(1,MAX_FILE_NUM+1):
    FILES.append("B_"+str(x))

In [56]:
#List of all images
images = []
images_label = []
for v in VARIETIES:
    for f in FILES:
        try:
            img = envi.open(exactPathHDR(v,f),exactPathBIL(v,f))
            images.append(img)
            images_label.append(v)
        except:
            pass

In [57]:
train_dataset = []
train_dataset_label = []
test_dataset = []
test_dataset_label = []

In [58]:
tic = start_timer()
for index, img in enumerate(images):
    count = 0
    label = images_label[index]
    while count<IMAGE_COUNT:
        aug_img = getAugumentedImage(img,BAND_NUMBER)
        
        if checkAugumentedImage(aug_img):
            aug_img = DL_Method(aug_img[:,:,FIRST_BAND:LAST_BAND+1])
            if count%5 == 0:
                test_dataset.append(aug_img)
                test_dataset_label.append(label)
            else:
                train_dataset.append(aug_img)
                train_dataset_label.append(label)
            count+=1  
            
    if TESTING:
        break
        
toc = end_timer()
show_time(tic,toc)

Testing started
Testing time (s) = 29.49391120002838



In [59]:
train_dataset = np.array(train_dataset)
train_dataset_label = np.array([VARIETIES_CODE[label] for label in train_dataset_label])
test_dataset = np.array(test_dataset)
test_dataset_label = np.array([VARIETIES_CODE[label] for label in test_dataset_label])

In [60]:
len(test_dataset)

8

In [61]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import pyplot as plt
# import math, sys, pdb, os
import keras
import keras.backend as K
import tensorflow as tf
from keras.layers import Input, Conv2D, MaxPooling2D, Activation, BatchNormalization, Add, Conv2DTranspose, Flatten, Dense, Conv1D, AveragePooling2D, LeakyReLU, PReLU, GlobalAveragePooling2D
from keras.layers.core import Dropout
from keras.layers import concatenate
from keras.models import Model

import os, pdb, timeit
import numpy as np
from keras.callbacks import TensorBoard
from keras.optimizers import Adam
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt

from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
import matplotlib.cm as cm
import cv2
from keras import activations
import vis
# from vis.visualization import visualize_saliency, overlay
# from vis.utils import utils

In [62]:
def normalizeDataWholeSeed(data,normalization_type='max'):
    
    if normalization_type == 'max':
        for idx in range(data.shape[0]):
            data[idx,:,:,:] = data[idx,:,:,:]/np.max(abs(data[idx,:,:,:]))
            
    elif normalization_type == 'l2norm':
        from numpy import linalg as LA
        for idx in range(data.shape[0]):
            data[idx,:,:,:] = data[idx,:,:,:]/LA.norm(data[idx,:,:,:]) # L2-norm by default        
        
    return data

In [63]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    import itertools
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.clim(0,sum(cm[0,:]))
    plt.xlabel('Predicted label')

In [64]:
def top_K_classification_accuracy(y_predicted, y_true, K=1):

    num_samples = y_predicted.shape[0]
    num_classes = y_predicted.shape[1]

    if K > num_classes:
        sys.exit(1)

    temp = np.zeros((num_samples,))

    for idx in range(num_samples):
        curr_predicted = np.argsort(y_predicted[idx,:])
        curr_predicted = curr_predicted[::-1] # descending

        if y_true[idx] in curr_predicted[:K]:
            temp[idx] = 1

    return 100.0 * np.sum(temp)/num_samples

In [65]:
def single_conv_block(x, kernel_size=3, activation_type='relu', num_filters=128,use_bias=False):
    # Batch norm
    x = BatchNormalization()(x)
    x = Conv2D(num_filters, kernel_size, activation=None, use_bias=use_bias, padding='same',
               kernel_initializer='truncated_normal')(x)

    # Activation
    x = Activation(activation_type)(x)
    return x

In [66]:
def conv2D_ResNet(x, kernel_size, activation_type, dropout_rate, num_filters_first_conv1D=[32,64,128]):

    x = single_conv_block(x,1,activation_type,num_filters_first_conv1D[0])

    x = single_conv_block(x,kernel_size,activation_type,num_filters_first_conv1D[1],use_bias=True)

    x = single_conv_block(x,1,activation_type,num_filters_first_conv1D[2])

    # Dropout
    return Dropout(dropout_rate)(x)

In [67]:
def createBlock_ResNet2D(x, num_layers, kernel_size, activation_type, dropout_rate, num_filters_first_conv1D):

    for idx_layer in range(num_layers):
        x = conv2D_ResNet(x, kernel_size, activation_type, dropout_rate, num_filters_first_conv1D)

    return x

In [80]:
def ResNet2D_classifier(data_num_rows, data_num_cols, num_classes, kernel_size=3, num_layers_each_block=[],
                        num_chan_per_block=[], activation_type='relu', dropout_rate=0.0, num_input_chans=1, num_nodes_fc=64):

    input_data = Input(shape=(data_num_rows, data_num_cols, num_input_chans))

    # Input layer: Conv2D -> activation
    x = Conv2D(num_chan_per_block[0][0], kernel_size, activation=None, use_bias=True, padding='same',
               kernel_initializer='truncated_normal')(input_data)

    # Activation
    x = Activation(activation_type)(x)


    #  Blocks & Downsampling Layers
    for idx_block in range(len(num_layers_each_block)):
        x = createBlock_ResNet2D(x, num_layers_each_block[idx_block], kernel_size, activation_type, dropout_rate,
                                 num_chan_per_block[idx_block])

        x = BatchNormalization()(x)

        if idx_block != len(num_layers_each_block)-1:
            x = Conv2D(num_chan_per_block[idx_block][1]*2, kernel_size, strides = 2, activation=None, use_bias=True, padding='valid',
                   kernel_initializer='truncated_normal')(x)
        else:
            x = GlobalAveragePooling2D()(x)

        x = Dropout(dropout_rate)(x)

    # Output layer
    x = BatchNormalization()(x)
    
    x = Flatten()(x)

    x = Dense(units=num_nodes_fc, activation=None, kernel_initializer='truncated_normal')(x)
#     x = Dense(units=32, activation=None, kernel_initializer='truncated_normal')(x)

    # Activation
    x = Activation(activation_type)(x)

    x = BatchNormalization()(x)
    output_data = Dense(units=num_classes, activation='softmax', kernel_initializer='truncated_normal')(x)

    return Model(inputs=input_data, outputs=output_data)

In [81]:
def evaluate(model,dataset,dataset_label,normalization_type):
    print("--------------Make Predictions--------------")    
    x = np.array(dataset)
    labels = np.array(dataset_label)
    
    # Normalize the data
    x = normalizeDataWholeSeed(x,normalization_type=normalization_type)
    
    num = x.shape[0]

    tic = start_timer()
    labels_predicted = model.predict(x)
    toc = end_timer()
    show_time(tic,toc)
    
    print(labels_predicted)
    print("--------")
    # Classification accuracy
    labels_integer_format = labels
    labels_predicted_integer_format = np.argmax(labels_predicted, axis=1)

    acc_top2 = top_K_classification_accuracy(labels_predicted, labels_integer_format, K=2)
    acc_top1 = top_K_classification_accuracy(labels_predicted, labels_integer_format, K=1)
    
    # Confusion matrices
    confusion_matrix_results = confusion_matrix(labels_integer_format, labels_predicted_integer_format)
    print("Confusion matrix = ")
    print(confusion_matrix_results)
    print("------------------------------------------------")

In [82]:
def predict(model,normalization_type):
    evaluate(model,train_dataset,train_dataset_label,normalization_type)
    
    evaluate(model,test_dataset,test_dataset_label,normalization_type)
    


In [83]:
def createAndTrainResNetB():
    
    learning_rate_base = LEARNING_RATE_BASE
    kernel_size = 3
    num_epochs = NUM_EPOCHS
    batch_size = BATCH_SIZE
    dropout_rate = 0.15 
    activation_type = 'relu'
    num_nodes_fc = 256
    wheat_types =  VARIETIES
    normalization_type = 'max'
    num_layers_each_block = [1, 1]
    num_chan_per_block = [[64,128,128],[128,128,256]]
    N_classes = len(wheat_types)
    
    ############ Load data ############
    print("--------------Load Data--------------")

    # Load training data and their corresponding labels
    x_training = np.array(train_dataset)
    labels_training = np.array(train_dataset_label)
    
    # Normalize the data
    x_training = normalizeDataWholeSeed(x_training,normalization_type=normalization_type)
    
    # Extract some information
    num_training = x_training.shape[0]
    N_spatial = x_training.shape[1:3]
    N_bands = x_training.shape[3]
    
    print("--------------Done--------------")
    
    ############ Create a model ############
    print("--------------Create a model--------------")
    
    # Generate a model
    model = ResNet2D_classifier(data_num_rows = N_spatial[0], 
                                data_num_cols = N_spatial[1], 
                                num_classes = N_classes,
                                kernel_size = kernel_size, 
                                num_layers_each_block = num_layers_each_block,
                                num_chan_per_block = num_chan_per_block, 
                                activation_type = activation_type,
                                dropout_rate = dropout_rate, 
                                num_input_chans = N_bands, 
                                num_nodes_fc = num_nodes_fc)

    # Compile the model
    adam_opt = Adam(learning_rate=LEARNING_RATE_BASE, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.01)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=adam_opt, metrics=['accuracy'])
    print("---------Completed ")
    return model

In [84]:
model = createAndTrainResNetB()

--------------Load Data--------------
--------------Done--------------
--------------Create a model--------------


In [85]:
for x in range(4):
    print("From epochs: ",20*x+1," to ",20+20*x)
    tic = start_timer()
    print(model.fit(x=train_dataset,y=train_dataset_label,batch_size=BATCH_SIZE, epochs=20, initial_epoch = 0, verbose=2,validation_split=0.40,shuffle=True))
    toc = end_timer()
    show_time(tic,toc)
    print("for testing")
    print(model.evaluate(test_dataset,test_dataset_label))
    print("for training")
    print(model.evaluate(train_dataset,train_dataset_label))

From epochs:  1  to  20
Testing started
Epoch 1/20
1/1 - 7s - loss: 0.9014 - accuracy: 0.5000 - val_loss: 0.6938 - val_accuracy: 0.0000e+00 - 7s/epoch - 7s/step
Epoch 2/20
1/1 - 0s - loss: 0.7059 - accuracy: 0.7500 - val_loss: 0.6944 - val_accuracy: 0.0000e+00 - 225ms/epoch - 225ms/step
Epoch 3/20
1/1 - 0s - loss: 0.7046 - accuracy: 0.5000 - val_loss: 0.6948 - val_accuracy: 0.0000e+00 - 220ms/epoch - 220ms/step
Epoch 4/20
1/1 - 0s - loss: 0.7589 - accuracy: 0.5000 - val_loss: 0.6942 - val_accuracy: 0.0000e+00 - 212ms/epoch - 212ms/step
Epoch 5/20
1/1 - 0s - loss: 0.7262 - accuracy: 0.2500 - val_loss: 0.6942 - val_accuracy: 0.0000e+00 - 217ms/epoch - 217ms/step
Epoch 6/20
1/1 - 0s - loss: 0.7023 - accuracy: 0.7500 - val_loss: 0.6936 - val_accuracy: 0.0000e+00 - 213ms/epoch - 213ms/step
Epoch 7/20
1/1 - 0s - loss: 0.7101 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.0000e+00 - 216ms/epoch - 216ms/step
Epoch 8/20
1/1 - 0s - loss: 0.7195 - accuracy: 0.5000 - val_loss: 0.6937 - va

Epoch 19/20
1/1 - 0s - loss: 0.6860 - accuracy: 0.5000 - val_loss: 0.7041 - val_accuracy: 0.0000e+00 - 205ms/epoch - 205ms/step
Epoch 20/20
1/1 - 0s - loss: 0.6752 - accuracy: 0.7500 - val_loss: 0.7045 - val_accuracy: 0.0000e+00 - 214ms/epoch - 214ms/step
Testing time (s) = 4.412202499981504

for testing
1/1 [==============================] - 0s 105ms/step - loss: 0.6933 - accuracy: 0.5000
[0.6933435201644897, 0.5]
for training
1/1 [==============================] - 0s 96ms/step - loss: 0.6930 - accuracy: 0.5000
[0.6929892897605896, 0.5]
From epochs:  61  to  80
Testing started
Epoch 1/20
1/1 - 0s - loss: 0.6644 - accuracy: 0.7500 - val_loss: 0.7047 - val_accuracy: 0.0000e+00 - 272ms/epoch - 272ms/step
Epoch 2/20
1/1 - 0s - loss: 0.6802 - accuracy: 0.5000 - val_loss: 0.7048 - val_accuracy: 0.0000e+00 - 191ms/epoch - 191ms/step
Epoch 3/20
1/1 - 0s - loss: 0.6838 - accuracy: 0.7500 - val_loss: 0.7041 - val_accuracy: 0.0000e+00 - 196ms/epoch - 196ms/step
Epoch 4/20
1/1 - 0s - loss: 0.6801